# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity
                        WITH REPLICATION = 
                        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try: 
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# The following creates the artist song table with sessionId, itemInSession, artist name
# song title, length with the primary key sessonId and itemInSession
query_1 = """CREATE TABLE IF NOT EXISTS artist_song(sessionId int, 
             itemInSession int, artist_name varchar, 
             song_title varchar, length float, 
             PRIMARY KEY (sessionId, itemInSession))"""
try:
    session.execute(query_1)
except Exception as e:
    print(e)

                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song(sessionId, itemInSession, artist_name, song_title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)
        #print(query)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# The following query selects artist name , song title and length from the artist_song table
# where the sessionId = 338 and itemInSession = 4
queryselect1 = "SELECT artist_name, song_title, length from artist_song WHERE sessionId = 338 and itemInSession = 4"
rows = session.execute(queryselect1)
for row in rows:
    print(row)
    


Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
file = 'event_datafile_new.csv'
#the following creates a table Song_Playlist_Session with artistname, userId, song title
# first name, last name, itemInSession and sessionid
query_2 = """CREATE TABLE IF NOT EXISTS song_playlist_session (artist_name varchar, 
             userId int, song_title varchar, firstName varchar, lastName varchar,
             itemInSession int, sessionId int, 
             PRIMARY KEY((userId,sessionId), itemInSession))"""
try:
    session.execute(query_2)
except Exception as e:
    print(e)
    
    
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        insertQuery = "INSERT INTO song_playlist_session(userId, sessionId, itemInSession, artist_name, song_title, firstName, lastName)"
        insertQuery = insertQuery + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(insertQuery, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)
                    

In [12]:
#The following query selects artist name song title first name and last name
#where the sessionId = 182 and the userId = 10 
selectQuery = "SELECT artist_name, song_title, firstName, lastName from song_playlist_session WHERE sessionId = 182 and userId = 10"
try:
    outputRows = session.execute(selectQuery)
except Exception as e:
    print(e)
    
for row in outputRows:
    print(row)

                    

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [14]:
#The following creates a User table with userId, first name, last name and song title with the Primary Key of
#song title and userId 
query_3 = """CREATE TABLE IF NOT EXISTS user(userId int, 
             firstName varchar, lastName varchar, song_title varchar, 
             PRIMARY KEY(song_title, userid))"""
file = 'event_datafile_new.csv'
try:
    session.execute(query_3)
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        insertQuery = "INSERT INTO user(song_title, userid, firstName, lastName) VALUES (%s, %s, %s, %s)"
        try:
            session.execute(insertQuery, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

    

In [15]:
#The following Query Selects from the User table where the Song Title is 'All Hands Against His Own'
selectQuery = "SELECT * FROM user WHERE song_title='All Hands Against His Own' "
try:
    outputRows=session.execute(selectQuery)
except Exception as e:
    print(e)
for row in outputRows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [16]:
#The following Query Drops the Artist Song Table
dropQuery = "DROP TABLE artist_song"
try:
    outputRows= session.execute(dropQuery)
except Exception as e:
    print(e)
#The following Query Drops the Song Playlist Session Table
dropQuery2 = "DROP TABLE song_playlist_session"
try:
    outputRows = session.execute(dropQuery2)
except Exception as e:
    print(e)
    
#The following Query Drops the User table 
dropQuery3 = "DROP TABLE user"
try:
    outputRows = session.execute(dropQuery3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()